In [26]:
from server.optimisation.models import *
from server.optimisation.gen_data import *
import numpy as np
from sklearn.linear_model import LinearRegression

MAX_DEFERABLES = 3
MPP = 4
TICK_LENGTH = 300 / 60
GAMMA = 0.99
STACKED_NUM = 10
MAX_FLYWHEEL_CAPACITY = 50
MAX_IMPORT_EXPORT = 100

ModuleNotFoundError: No module named 'sklearn'

In [19]:
def getTicksForDay(id):
    day = Day.model_validate({"day": id, "deferables": getDefDemands(id)})
    ticks = []
    for tick in range(TICKS_PER_DAY):
        sun = getSunlight(tick)
        demand = getInstDemand(id, tick)
        sell, buy = getPrice(id, tick)

        ticks.append(
            Tick.model_validate(
                {
                    "tick": tick,
                    "sun": sun,
                    "demand": demand,
                    "sell_price": sell,
                    "buy_price": buy,
                    "day": id,
                }
            )
        )
    return day, ticks

def simulate_day_naive(day, ticks, use_flywheel=False, export_extra=False):
    costs = []
    flywheel_amt = 0
    for tick in ticks:
        power = (tick.sun / 100) * MPP 
        sun_energy = power * TICK_LENGTH # sun in percentage to joules per tick
        
        total_energy = sun_energy

        if use_flywheel:
            total_energy += flywheel_amt
            flywheel_amt = 0

        total_energy -= tick.demand

        for deferable in day.deferables:
            if deferable.start == tick.tick:
                total_energy -= deferable.energy
        
        if total_energy < 0:
            cost = -total_energy * tick.sell_price
        elif use_flywheel:
            flywheel_amt = min(MAX_FLYWHEEL_CAPACITY, flywheel_amt + total_energy)
        elif export_extra: 
            cost = total_energy * tick.buy_price # Export extra energy
        
        costs.append(cost)
    
    return sum(costs)


In [21]:
day, tick = getTicksForDay(0)
day

Day(day=0, deferables=[Deferable(start=0, end=59, energy=50.0), Deferable(start=48, end=59, energy=28.098682749008287), Deferable(start=38, end=48, energy=24.321974987799532)])

In [25]:
# model = LinearRegression()

# model.fit()



NameError: name 'LinearRegression' is not defined

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    # input layer -> 4 variables
    # hidden layer 1-n
    # output -> classify between 3 different possible output

    def __init__(self, in_vars=6, h1=18, h2=18, out_vars=6):
        super().__init__()
        # fully connected layer 1
        self.fc1 = nn.Linear(in_vars, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_vars)

    def forward(self, x):
        # if < return 0
        # else gives the val
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [46]:
model = Model(6, 100, 100, 6)

In [47]:
ticks = []
for i in range(1000):
    _, tmp_ticks = getTicksForDay(i)
    for tmp in tmp_ticks: ticks.append([tmp.day, tmp.tick, tmp.demand, tmp.sun, tmp.buy_price, tmp.sell_price])

ticks 

[[0, 0, 1.4417154046806644, 0, 26, 53],
 [0, 1, 1.7881847531554629, 0, 30, 60],
 [0, 2, 2.838166736175902, 0, 40, 81],
 [0, 3, 0.5947080382873042, 0, 18, 36],
 [0, 4, 0.5408559447931041, 0, 17, 35],
 [0, 5, 0.0, 0, 5, 11],
 [0, 6, 0.9996755931358106, 0, 22, 44],
 [0, 7, 0.2441197115523996, 0, 14, 29],
 [0, 8, 0.8734151696239874, 0, 21, 42],
 [0, 9, 0.0, 0, 8, 16],
 [0, 10, 0.0, 0, 7, 15],
 [0, 11, 0.0, 0, 8, 17],
 [0, 12, 0.0, 0, 10, 21],
 [0, 13, 0.8539810137804718, 0, 27, 55],
 [0, 14, 2.319479936120244, 0, 44, 89],
 [0, 15, 1.389484772815805, 0, 37, 74],
 [0, 16, 0.6621143785125555, 10, 27, 55],
 [0, 17, 0.0, 20, 15, 31],
 [0, 18, 2.316038215368986, 30, 38, 77],
 [0, 19, 1.0650371593315235, 40, 23, 46],
 [0, 20, 3.262751522893784, 49, 43, 86],
 [0, 21, 2.779245168944624, 58, 33, 67],
 [0, 22, 2.531150942518961, 66, 27, 54],
 [0, 23, 4.1699867207380645, 74, 39, 79],
 [0, 24, 1.5513947296208364, 80, 10, 21],
 [0, 25, 0.3632761536945244, 86, 5, 10],
 [0, 26, 1.9885432653133743, 91, 9, 

In [48]:
X_test = torch.FloatTensor(ticks[:-1])
Y_test = torch.FloatTensor(ticks[1:])

In [49]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [50]:
# training
epochs = 2000
losses = []

for i in range(epochs):
    # send data forward
    y_pred = model.forward(X_test)

    loss = criterion(y_pred, Y_test)

    if i % 10 == 0: print(f'epoch: {i} and loss: {loss}')

    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch: 0 and loss: 40653.41015625
epoch: 10 and loss: 32203.060546875
epoch: 20 and loss: 6826.51025390625
epoch: 30 and loss: 2321.830078125
epoch: 40 and loss: 1022.6607666015625
epoch: 50 and loss: 620.0254516601562
epoch: 60 and loss: 558.3306274414062
epoch: 70 and loss: 534.9961547851562
epoch: 80 and loss: 510.0788879394531
epoch: 90 and loss: 505.58502197265625
epoch: 100 and loss: 500.2809753417969
epoch: 110 and loss: 498.09808349609375
epoch: 120 and loss: 496.53460693359375
epoch: 130 and loss: 495.35345458984375
epoch: 140 and loss: 494.3246154785156
epoch: 150 and loss: 493.44842529296875
epoch: 160 and loss: 492.68341064453125
epoch: 170 and loss: 491.99835205078125
epoch: 180 and loss: 491.36859130859375
epoch: 190 and loss: 490.7801818847656
epoch: 200 and loss: 490.226318359375
epoch: 210 and loss: 489.7016906738281
epoch: 220 and loss: 489.20355224609375
epoch: 230 and loss: 488.7307434082031
epoch: 240 and loss: 488.28326416015625
epoch: 250 and loss: 487.8588256835